In [4]:
import pyprind
import pandas as pd
import os

"""basepath = 'aclImdb'

labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()"""

"basepath = 'aclImdb'\n\nlabels = {'pos' : 1, 'neg' : 0}\npbar = pyprind.ProgBar(50000)\ndf = pd.DataFrame()"

In [5]:
"""for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']"""

"for s in ('test', 'train'):\n    for l in ('pos', 'neg'):\n        path = os.path.join(basepath, s, l)\n        for file in sorted(os.listdir(path)):\n            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:\n                txt = infile.read()\n            df = df.append([[txt, labels[l]]], ignore_index=True)\n            pbar.update()\n\ndf.columns = ['review', 'sentiment']"

In [6]:
import numpy as np
np.random.seed(0)
"""df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')"""

"df = df.reindex(np.random.permutation(df.index))\ndf.to_csv('movie_data.csv', index=False, encoding='utf-8')"

In [7]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.shape

(50000, 2)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()#создаем 1-граммы, чтобы создать 2-граммы передаем ngram_range = (2, 2)
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
count.vocabulary_#глосарий - слово и индекс

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

In [9]:
count = CountVectorizer(ngram_range = (2, 2))
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
count.vocabulary_

{'the sun': 9,
 'sun is': 7,
 'is shining': 1,
 'the weather': 10,
 'weather is': 11,
 'is sweet': 2,
 'shining the': 6,
 'sweet and': 8,
 'and one': 0,
 'one and': 4,
 'one is': 5,
 'is two': 3}

In [10]:
bag.toarray()#вектора признаков, сырые частоты термов 
#tf(t, d) - сколько раз терм t встречается в документе d

array([[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [11]:
#мера частота терма - обратная частота документа позволяет найти часто встречающиеся слова
#которые не имеют смысла tf-idf (t, d) = tf(t, d) * idf(t, d)
#idf(t, d) = log(nd/(1 + df(t, d))) nd - общее количество документов
#df(t, d) - количество документов d содержащих терм t

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',#нормализуем вектор термов деля на L2 норму
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.58 0.   0.   0.   0.   0.   0.58 0.   0.58 0.   0.  ]
 [0.   0.   0.58 0.   0.   0.   0.   0.   0.   0.   0.58 0.58]
 [0.57 0.22 0.22 0.28 0.28 0.28 0.28 0.22 0.28 0.22 0.22 0.22]]


In [13]:
#в sklearn формулы отличаются, а именно:
#idf(t, d) = log(1 + nd/(1 + df(t, d)))
#tf-idf (t, d) = tf(t, d) * (idf(t, d) + 1) из-за того что smooth_idf=True чтобы  = 0 для термов встречающихся во всех документах


In [14]:
df.loc[0, 'review'][-50:] #данные необходимо очитстить

'is seven.<br /><br />Title (Brazil): Not Available'

In [16]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #удаление html разметки
    emoticons = re.findall('(?::|;|=)(?:-)?(?:-\)|\(|D|Р)',
                           text)#находми все эмотиконы
    text = (re.sub('[\W]+', ' ', text.lower()) + #находим и удалям все несловарные символы
            ' '.join(emoticons).replace('-', ''))#возвращаем эмотиконы
    return text
    

In [17]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [18]:
df['review'] = df['review'].apply(preprocessor)

In [19]:
def tokenizer(text):#один из вариантов разбиения на лексемы
    return(text.split())
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [20]:
#аналог - стемминг - преобразование слова в его корневую форму
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [21]:
#удаление стоп-слов - слова, которые распространены в тексте и вероятно не несут в себе полезной информации
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('runners like running and thus they run')[-10:]
 if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

In [22]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
               'clf__penalty': ['ll', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                    tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm' : [None],
               'clf__penalty': ['11', '12'],
               'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf',
                      LogisticRegression(random_state=0,
                                         solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=2,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
